In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet152
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy

C:\Users\POOJYANTH REDDY\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\POOJYANTH REDDY\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def load_and_process_images(path_dir, class_labels, target_num_images):
    images = []
    labels = []
    min_images = 99999999
    class_images = {}

    for label, class_name in enumerate(class_labels):
        class_dir = os.path.join(path_dir, class_name)
        class_images[class_name] = []

        if not os.path.exists(class_dir):
            print(f"Directory '{class_name}' not found in '{path_dir}'. Skipping...")
            continue

        for jpg in os.listdir(class_dir):
            image_path = os.path.join(class_dir, jpg)
            image_high_resolution = cv2.imread(image_path)

            if image_high_resolution is None:
                print(f"Could not read image '{jpg}' in '{class_name}' directory. Skipping...")
                continue

            # print(f"Reading image '{jpg}' in '{class_name}' directory...")
            image_change_color = cv2.cvtColor(image_high_resolution, cv2.COLOR_BGR2RGB)
            image_low_resolution = cv2.resize(image_change_color, (256, 256))
            class_images[class_name].append(image_low_resolution)
            min_images = min(min_images, len(class_images[class_name]))
        print(f"Class '{class_name}' has {len(class_images[class_name])} images.")


    for class_name, images_list in class_images.items():
            images.extend(images_list)
            labels.extend([class_name] * len(images_list))

    return np.asarray(images), np.asarray(labels), min_images, class_images



dataset_directory = './Datasets/DataImages/'
class_labels = ['Catla', 'Cyprinus carpio', 'Grass Carp', 'Mori', 'Rohu', 'Silver']
target_num_images = 50

# images, labels, min_images, class_images = load_and_process_images(dataset_directory, class_labels, target_num_images)

# print(f"Number of images: {len(images)}")
# print(f"Number of labels: {len(labels)}")


In [8]:
# Constants
BATCH_SIZE = 64
TAU = 0.1
PROJECTION_DIM = 128
EPOCHS = 10

# Image shape
IMG_SHAPE = (256, 256, 3)

# Load and preprocess data
images, labels, _, _ = load_and_process_images(dataset_directory, class_labels, target_num_images)

# Split data into train and test sets
images_train, images_test, labels_train, labels_test = train_test_split(images, labels, test_size=0.2, random_state=42)

Class 'Catla' has 20 images.
Class 'Cyprinus carpio' has 50 images.
Class 'Grass Carp' has 11 images.
Class 'Mori' has 70 images.
Class 'Rohu' has 73 images.
Class 'Silver' has 47 images.


In [9]:
# Split the data into train and test sets
split_ratio = 0.8  # 80% train, 20% test
num_samples = len(images)
split_index = int(split_ratio * num_samples)

train_images, test_images = images[:split_index], images[split_index:]
train_labels, test_labels = labels[:split_index], labels[split_index:]

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(buffer_size=num_samples).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(BATCH_SIZE)

In [13]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Define the encoder architecture
def get_encoder(input_shape, projection_dim=128):
    base_model = tf.keras.applications.ResNet50(weights=None, include_top=False, input_shape=input_shape)
    base_model.trainable = True

    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=True)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(projection_dim, activation='relu')(x)
    x = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))(x)

    return Model(inputs, x)

# Define the contrastive loss function
def contrastive_loss(z_i, z_j, temperature=0.1):
    sim_ij = tf.reduce_sum(tf.multiply(z_i, z_j), axis=-1)
    exp_sim_ij = tf.exp(sim_ij / temperature)
    numerator = tf.linalg.diag_part(exp_sim_ij)
    denominator = tf.reduce_sum(exp_sim_ij, axis=-1)
    loss = -tf.math.log(numerator / denominator)
    return loss

# Create the SimCLR model
def get_simclr_model(encoder, temperature=0.1):
    inputs = tf.keras.Input(shape=(None, None, 3))
    x_i = encoder(inputs)
    x_j = encoder(inputs)

    simclr_model = Model(inputs, [x_i, x_j])
    simclr_model.compile(optimizer=Adam(), loss=lambda _, y_pred: contrastive_loss(*y_pred, temperature=temperature))

    return simclr_model

# Dummy data augmentation function
def data_augmentation(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    return image

# SimCLR training loop
def train_simclr(simclr_model, dataset, epochs=10, batch_size=64):
    for epoch in range(epochs):
        for batch in dataset:
            images_i = data_augmentation(batch[0])  # Assuming the image data is in the first element of the tuple
            images_j = data_augmentation(batch[0])  # Assuming the image data is in the first element of the tuple

            with tf.GradientTape() as tape:
                z_i, z_j = simclr_model(images_i, training=True)

                # Create negative pairs (z_i, z_k)
                shuffled_indices = tf.range(tf.shape(images_i)[0])
                shuffled_indices = tf.random.shuffle(shuffled_indices)
                images_k = tf.gather(images_i, shuffled_indices)
                z_k = simclr_model(images_k, training=True)[0]

                # Calculate contrastive loss
                loss = contrastive_loss(z_i, z_j) + contrastive_loss(z_i, z_k)

            gradients = tape.gradient(loss, simclr_model.trainable_variables)
            simclr_model.optimizer.apply_gradients(zip(gradients, simclr_model.trainable_variables))

# Example usage
input_shape = (224, 224, 3)
encoder = get_encoder(input_shape)
simclr_model = get_simclr_model(encoder)

# Assuming 'dataset' is your training dataset
train_simclr(simclr_model, train_dataset, epochs=10, batch_size=64)


ValueError: Exception encountered when calling layer 'model_9' (type Functional).

Input 0 of layer "model_8" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(64, 256, 256, 3)

Call arguments received by layer 'model_9' (type Functional):
  • inputs=tf.Tensor(shape=(64, 256, 256, 3), dtype=uint8)
  • training=True
  • mask=None